# Investments notebook

This notebook is focused in going through your investments gains and how this are going.

In [ ]:
LOAD_CONFIGS = {
    'data_path': "./data", # Directory that contains the expenses data
    'nubank_file_pattern': "nubank*.csv",
    'splitwise_groups': [
        'jacas-keter',
        '2020-mozi-e-eu',
        '2019-mozi-e-eu',
        '2018-mozi-e-eu'
    ],
    'manual_file_pattern': "Extrato outras contas*Despesas*.csv",
    'incomes_file_pattern': "Extrato outras contas*Rendimentos*.csv",
    'person_who_pays': "Lucas Alencar", # Person name that the expenses will be extracted from Splitwise csv
}

from datetime import date, timedelta
from date_helpers import previous_month

# Date used to make month analysis through the notebook
BASE_DATE = previous_month(date.today())

# Period used for analysis
BASE_PERIOD_START = BASE_DATE - timedelta(days=365)
BASE_PERIOD_END = BASE_DATE

# Year you were born
YEAR_OF_BIRTH = 1991

## Data load and preprocess
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import display
import record_summary as rs

from load import investments
invest = investments.load(**LOAD_CONFIGS)

# Month returns

In [ ]:
from investments import monthly as monthly_investments
from load import central_bank

# BASE_DATE = date(2019, 11, 1)
print(">>> Month of analysis:", BASE_DATE.month, BASE_DATE.year)

current_ipca = central_bank.ipca_for_month(BASE_DATE)
current_ipca = current_ipca * 100 if current_ipca else None
print("> Current IPCA:", current_ipca, "%")

RETURN_FOR_MONTH_GOAL = 0.005
RETURN_WITH_INFLATION_GOAL = 0.0045

# Table with returns for the current month
summary_invest = monthly_investments.summary(invest, BASE_DATE).sort_values('Return for month (%)', ascending=False)

display(monthly_investments.style_summary(summary_invest, RETURN_FOR_MONTH_GOAL, RETURN_WITH_INFLATION_GOAL))

# Return over time

In [ ]:
# Get all investments titles
# import pandas as pd
# display(pd.DataFrame(invest['title'].unique(), columns=['title']))

# Some visualizations to use and see the return history for the groups
general = ['Nuconta Renda Fixa 100% CDI',
           'Tesouro Selic 2021',
           'CDB Banco Maxima',
           'LCA Banco Maxima',
           'CDB Banco Indusval',
           'TARPON GT FIC FIA',
           'Magnetis Diversificação Renda Fixa',
           'Magnetis Diversificação Multimercados',
           'Magnetis Diversificação Ações']

# Other investments in renda fixa are better than Tesouro Selic? They are still worth it?
renda_fixa = ['Nuconta Renda Fixa 100% CDI',
              'Tesouro Selic 2021',
              'Tesouro Selic 2023',
              'Magnetis Diversificação Renda Fixa',
              'CDB Banco Maxima',
              'LCA Banco Maxima',
              'CDB Banco Indusval',
              'Brasil Plural Yield Fundo de Investimento Renda Fixa Referen']

# Is Magnetis investing in good stocks? Is the cumulative return increasing?
renda_variavel = ['Magnetis Diversificação Ações',
                  'BRAX11',
                  'TARPON GT FIC FIA',
                  'Magnetis Diversificação Multimercados']

# Does bitcoin is still a good experiment?
bitcoins = ['Bitcoin', 'Cripto Trade']

# There are better opportunities of investments over these Tesouros?
tesouros = ['Tesouro IPCA+ 2024',
            'Tesouro Selic 2021',
            'Tesouro Selic 2023']

RETURN_STARTING_MONTH = BASE_PERIOD_START.strftime('%Y-%m')
RETURN_ENDING_MONTH = BASE_PERIOD_END.strftime('%Y-%m')

print('from:', RETURN_STARTING_MONTH, 'to:', RETURN_ENDING_MONTH)

PLOT_TITLE_GROUP = general

from plotting import over_time as plot_over_time
from investments import over_time

plot_over_time.plot(over_time.return_percentage_over_time(invest)\
                        .loc[RETURN_STARTING_MONTH:RETURN_ENDING_MONTH, PLOT_TITLE_GROUP], 
               title='Return over time (%)')

plot_over_time.plot(over_time.cumulative_return_over_time(invest)\
                        .loc[RETURN_STARTING_MONTH:RETURN_ENDING_MONTH, PLOT_TITLE_GROUP], 
               title='Cumulative return over time')

# Monthly view

In [ ]:
from investments import assets

ASSETS_SUMMARY_START_DATE = BASE_PERIOD_START
ASSETS_SUMMARY_END_DATE = BASE_PERIOD_END

assets_summary = assets.summary(invest, ASSETS_SUMMARY_START_DATE, ASSETS_SUMMARY_END_DATE)
display(assets.style_summary(assets_summary))

from investments import distribution
distribution.plot(invest, 'type', BASE_DATE)

# Plot line to see assets progress
from plotting import over_time
over_time.plot(assets_summary['Return'].to_frame(), title='Return')
over_time.plot(assets_summary['Applications'].to_frame(), title='Applications')

# Liquidations

In [ ]:
from investments import post_liquidation
## TODO Add ROI to this table
post_liquidation.final_return(post_liquidation.liquidations_incomes(invest))

# Assets goals

### (PMS) Patrimônio Mínimo de Sobrevivência

Reserva de emergência

```6 * Custo mensal```

### (PMR) Patrimônio Mínimo Recomendado para segurança 

Possibilita decisões mais arriscada sobre carreira ou mudanças.

```12 * Custo mensal```

### (PI) Patrimônio Ideal para idade e consumo 

Dado os meus gastos mensais atuais, se eu continuar trabalhando até a aposentadoria (65 anos), eu conseguiria viver apenas gastando esse dinheiro.

```0.1 * PMR * Idade```

### (PNIF) Patrimônio Necessário para a Independência Financeira

Dado os meus rendimentos com investimentos, eu conseguiria manter o mesmo padrão apenas com o dinheiro investido

```PMR / Rendimento anual %```

In [ ]:
from investments import assets

ASSETS_GOALS_START_DATE = BASE_DATE - timedelta(days=365)
ASSETS_GOALS_END_DATE = BASE_DATE

print("Projected return for a year: {:,.4f}%"\
      .format(assets.projected_return_for_year(invest, ASSETS_GOALS_START_DATE, ASSETS_GOALS_END_DATE) * 100))

from load import expenses
expenses = expenses.load(**LOAD_CONFIGS)
expenses = expenses[expenses.category != 'renda']

expenses_by_month = rs.total_amount_by(rs.groupby_month(expenses), expenses)

goals = assets.goals(expenses_by_month,
                     invest, 
                     ASSETS_GOALS_START_DATE,
                     ASSETS_GOALS_END_DATE, 
                     YEAR_OF_BIRTH)

display(assets.style_goals(goals))

# Define your assets goals and how to distribute them
INVESTMENTS_GOAL = {
    'Emergência': goals.loc['PMS'].amount,
    'Aposentadoria': goals.loc['PI'].amount,
    'Experimento': 3000
}

invest_goals = assets.investment_goals(rs.records_for_month(invest, ASSETS_GOALS_END_DATE), 
                                       INVESTMENTS_GOAL)
assets.investment_goals_plot(invest_goals)

# Plot assets over time to see the progression
from plotting import over_time
over_time.plot(assets_summary.loc[:ASSETS_GOALS_END_DATE.strftime('%Y-%m'), 'Total'].to_frame(), 
               title='Assets')